In [1]:
import networkx as nx
import pandas as pd
import pickle
import math
from neo4j import GraphDatabase, Transaction

In [23]:
from src.ifc_to_nx_explorer import IfcToNxExplorer

In [25]:
with GraphDatabase.driver(
        "bolt://localhost:7688",
        auth=("neo4j", "23109900")) as driver:
    driver.verify_connectivity()
    val = driver.session().run("match (n) return n;").value()
val

[<Node element_id='4:246cdb52-7a29-4bfb-88de-dcab67f07ec3:0' labels=frozenset({'Rest'}) properties={'name': 'Evening'}>]

In [2]:
with GraphDatabase.driver(
        "bolt://localhost:7687",
        auth=("neo4j", "23109900")) as driver:
    driver.verify_connectivity()
    val = driver.session().run("match (n) return n;").value()
val

[<Node element_id='4:f2cab372-6b02-4aac-b4b8-2574ddf13184:0' labels=frozenset({'Work'}) properties={'name': 'VK'}>]

In [26]:
import os
os.getcwd()

'C:\\Users\\naumo\\PycharmProjects\\IFC_test\\notebooks'

In [15]:
G = pickle.load(open('../src/new_AR.pickle', 'rb'))

In [8]:
G = pickle.load(open('G_coord.pickle', 'rb'))

In [28]:
group_driver = GraphDatabase.driver(
    "bolt://localhost:7688",
    auth=("neo4j", "23109900")
)
group_driver.verify_connectivity()

In [101]:
# def add_node(tx: Transaction, id_, stor_id_, type_, name, coords,):
#     q_create = "CREATE (a:Element {id: $id, stor_id: $stor_id, type: $type, name: $name, coordinates: $crd})"
#     tx.run(q_create, id=str(id_), stor_id=stor_id_, type=type_, name=name, crd=coords)
def add_node(tx: Transaction, id_, stor_id_, props_):
    q = """
    CREATE (n:Element)
    SET n = $props
    SET n.id = $id
    SET n.stor_id = $stor_id
    """
    tx.run(q, id=str(id_), stor_id=str(stor_id_), props=props_)

def add_edge(tx: Transaction, n, m):
    q_edge = '''
    MATCH (a:Element) WHERE a.id = $id1
    MATCH (b:Element) WHERE b.id = $id2
    MERGE (a)-[r:CONTAINS]->(b)
    '''
    tx.run(q_edge, id1=str(n), id2=str(m))

def add_ifc_class(tx: Transaction, storey_id, class_id, class_name):
    q_class = '''
    MATCH (s:Element) WHERE s.id = $s_id
    MERGE (s)-[r:CONTAINS]->(:IfcClass {id: $id, name: $name})
    '''
    tx.run(q_class, s_id=str(storey_id), id=str(class_id), name=class_name)

def add_el_to_class(tx: Transaction, class_id, element_id):
    q_cls_el = '''
    MATCH (c:IfcClass) WHERE c.id = $cls_id
    MATCH (e:Element) WHERE e.id = $el_id
    MERGE (c)-[:CONSISTS_OF]->(e)
    '''
    tx.run(q_cls_el, cls_id=str(class_id), el_id=str(element_id))

def traverse(tx:Transaction, id1, id2):
    q_traverse = '''
    MATCH (a:Element) WHERE a.id = $id1
    MATCH (b:Element) WHERE b.id = $id2
    MERGE (a)-[:TRAVERSE]->(b)
    '''
    tx.run(q_traverse, id1=str(id1), id2=str(id2))

def add_class_rel(tx: Transaction, pred_name: str, flw_name: str):
    q_rel = '''
    MATCH (a:IfcClass) WHERE a.name = $name1
    MATCH (b:IfcClass) WHERE b.name = $name2
    MERGE (a)-[r:FOLLOWS]->(b)
    '''
    tx.run(q_rel, name1=pred_name, name2=flw_name)

In [145]:
classes = (
    'IfcWall',
    # 'IfcDoor',
    'IfcBuildingElementProxy',
    # 'IfcWindow',
    'IfcSlab',
    "IfcFlowTerminal",
    "IfcFurniture",
    'IfcCurtainWall',)
# Create group data
def add_class(tx: Transaction, class_name: str):
    q_class = '''
    MERGE (n:IfcClass {name: $name})
    '''
    tx.run(q_class, name=class_name)

with group_driver.session() as session:
    session.run('MATCH (n) DETACH DELETE n')
    for i in classes:
        session.execute_write(add_class, i)

    session.execute_write(add_class_rel, 'IfcBuildingElementProxy', 'IfcWall')
    session.execute_write(add_class_rel, 'IfcBuildingElementProxy', 'IfcSlab')
    session.execute_write(add_class_rel, 'IfcWall', 'IfcWindow')
    session.execute_write(add_class_rel, 'IfcWall', "IfcFlowTerminal")
    session.execute_write(add_class_rel, 'IfcWall', 'IfcCurtainWall')
    session.execute_write(add_class_rel, 'IfcWall', "IfcFurniture")
    session.execute_write(add_class_rel, 'IfcBuildingElementProxy', 'IfcDoor')
    session.execute_write(add_class_rel, 'IfcDoor', 'IfcWindow')

In [73]:
element_driver = GraphDatabase.driver(
    "bolt://localhost:7687",
    auth=("neo4j", "23109900")
)
element_driver.verify_connectivity()
# element_driver.verify_authentication()

In [154]:
with element_driver.session(database="neo4j") as session:
    session.run('MATCH (n) DETACH DELETE n')

    build_id = [node for node, data in G.nodes(data=True) if data.get('is_a') == 'IfcBuilding'][0]
    # print(build_id)
    session.execute_write(add_node, build_id, None, G.nodes[build_id])
                          # None,
                          # G.nodes[build_id].get('is_a', None),
                          # G.nodes[build_id].get('name', None),
                          # G.nodes[build_id].get("coordinates", 0),
                          # )

    k = 0
    for stor_id in G.successors(build_id):
        k += 1
        data = G.nodes[stor_id]
        if data.get('is_a') != 'IfcBuildingStorey':
            continue

        session.execute_write(add_node, stor_id, None, data)
                              # None, data['is_a'], data['name'], data['coordinates'])
        session.execute_write(add_edge, build_id, stor_id)


        contained_classes = set(G.nodes[i]['is_a'] for i in list(filter(
                lambda el_id: G.nodes[el_id]["coordinates"],
                G.successors(stor_id))
            ))
        cls_to_id = dict()
        for j, cls_name in enumerate(contained_classes):
            cls_to_id[cls_name] = str(stor_id) + '_' + str(j)
            session.execute_write(add_ifc_class, stor_id, cls_to_id[cls_name], cls_name)

        for cls in contained_classes:
            elements = list(filter(
                lambda el_id: G.nodes[el_id]["is_a"] == cls and G.nodes[el_id]["coordinates"],
                G.successors(stor_id))
            )

            # Для упорядочивания элементов одной группы
            start_point = (0, 0)
            def compute_angle(el_id):
                coordinates = G.nodes[el_id]["coordinates"]
                x, y, z = coordinates[0], coordinates[1], coordinates[2]
                angle = math.atan2(y - start_point[1], x - start_point[0])
                return z, angle

            for j, i in enumerate(sorted(elements, key=compute_angle)):
                s_data = G.nodes[i]
                if s_data["coordinates"]:
                    session.execute_write(add_node, i, stor_id, s_data)
                                          # s_data['is_a'], s_data['name'], s_data["coordinates"])
                    session.execute_write(add_el_to_class, cls_to_id[cls], i)
                    if j > 0:
                        session.execute_write(traverse, prev_id, i)
                    prev_id = i
                    if j == 4:
                        break

            q_rel = '''
            MATCH (a:IfcClass)-[r:FOLLOWS]->(b:IfcClass)
            RETURN a.name AS type1, b.name AS type2
            '''
            result = pd.DataFrame(group_driver.session().run(q_rel).data())
            result.apply(
                lambda row: connect_chains(stor_id, row.type1, row.type2),
                # lambda row: print(row.type1, row.type2),
                axis=1
            )

        if k == 4:
            break

163 IfcBuildingElementProxy 2349858 IfcWall 2161316
<class 'str'>
MATCH (a:Element) WHERE a.id = '2349858'
        MATCH (b:Element) WHERE b.id = '2161316'
        MERGE (a)-[r:TRAVERSE]->(b)
        
169 IfcBuildingElementProxy 2349337 IfcWall 719728
<class 'str'>
MATCH (a:Element) WHERE a.id = '2349337'
        MATCH (b:Element) WHERE b.id = '719728'
        MERGE (a)-[r:TRAVERSE]->(b)
        
175 IfcBuildingElementProxy 2316551 IfcWall 749004
<class 'str'>
MATCH (a:Element) WHERE a.id = '2316551'
        MATCH (b:Element) WHERE b.id = '749004'
        MERGE (a)-[r:TRAVERSE]->(b)
        
179 IfcBuildingElementProxy 3198133 IfcSlab 702965
<class 'str'>
MATCH (a:Element) WHERE a.id = '3198133'
        MATCH (b:Element) WHERE b.id = '702965'
        MERGE (a)-[r:TRAVERSE]->(b)
        
179 IfcBuildingElementProxy 3198133 IfcWall 764843
<class 'str'>
MATCH (a:Element) WHERE a.id = '3198133'
        MATCH (b:Element) WHERE b.id = '764843'
        MERGE (a)-[r:TRAVERSE]->(b)
        
179

In [153]:
def connect_chains(stor_id, type1: str, type2: str):
    q_get_last = f'''MATCH (s:Element {{is_a: '{type1}', stor_id: '{str(stor_id)}' }})
    WHERE NOT (s)-[]->(:Element {{is_a: '{type1}' }})
    RETURN s.id AS id
    LIMIT 1'''
    last_res = element_driver.session().run(q_get_last).data()
    q_get_first = f'''MATCH (s:Element {{is_a: '{type2}', stor_id: '{str(stor_id)}' }})
    WHERE NOT (:Element {{is_a: '{type2}' }})-[]->(s)
    RETURN s.id AS id
    LIMIT 1'''
    first_res = element_driver.session().run(q_get_first).data()

    if len(last_res) > 0 and len(first_res) > 0:
        q_rel = f'''MATCH (a:Element) WHERE a.id = '{last_res[0]['id']}'
        MATCH (b:Element) WHERE b.id = '{first_res[0]['id']}'
        MERGE (a)-[r:TRAVERSE]->(b)
        '''
        element_driver.session().run(q_rel)

In [ ]:
with group_driver.session() as session:
    q_rel = '''
    MATCH (a:IfcClass)-[r:FOLLOWS]->(b:IfcClass)
    RETURN a.name AS type1, b.name AS type2
    '''
    result = pd.DataFrame(session.run(q_rel).data())
    result.apply(
        lambda row: connect_chains(row.type1, row.type2),
        # lambda row: print(row.type1, row.type2),
        axis=1
    )
    print(result)

In [102]:
connect_chains(163, 'IfcBuildingElementProxy', 'IfcWall')